# Support Vector Machine

This notebook contains a from-scratch implementation of a Support Vector Machine (SVM) binary
classifier. This implementation only uses basic Python libraries like numpy and pandas, aside from
some basic scikit-learn functionality for loading and managing test datasets.

Joseph Melby,
2023

### Basics


This supervised learning algorithm constructs a hyperplane in $n$-dimensional space that corresponds
to a decision boundary of a binary classification problem. It does this by initializing and then
optimizing a set of feature weights and finding an intercept (parameters for a hyperplane). In
particular, we begin with a feature matrix $X$ of dimension $n$ and vector $y$ such that

- each row $X_i$ of $X$ contains features of the data set, and 
- each element $y_i$ of $y$ is a positive or negative number giving the feature vector class. 

#### Hyperplane Selection


The model is optimized to find a set of weights $W$ and an intercept $b$ defining a
hyperplane

$$W\cdot X + b$$

such that the function 

$$f(X_i) = sign(W\cdot self.features_i + b)$$

gives the model's classification of feature vector $X_i$ for each row of $X$. The hyperplane
parameters are optimized by both minimizing a cost function and maximizing the margin of separation
of the plane.

As a convenience for this implementation, we can push the intercept $b$ into the weights $W$ by just
adding them in as an extra column to the end of $W$:

$$ f(X_i) = \tilde{W} \cdot \tilde{X_i} + W_0 = \mathbf{W} \cdot X_i $$

where 

$$ \mathbf{W} = \left(\tilde{W}, W_0 \right), \qquad X_i = \left( \tilde{X_i}, 1 \right). $$

#### Cost Function


We have two options for the cost function, each with parameters that affect the model in slightly
different contexts. The first is 

$$ J(W) = \frac{1}{2}||W||^2 + C \left[ \frac{1}{n} \sum_{i=1}^{n} max \left(0, 1 - y_i \left(W
\cdot X_i
+ b \right)\right)\right] $$

Here, $C$ is affected while training the model; a larger $C$ corresponds to a narrow margin between
the hyperplane and support vector. The second option is 

$$ J(W) = \frac{\lambda}{2}||W||^2 + \frac{1}{n} \sum_{i=1}^{n} max(0, 1 - y_i(W \cdot X_i + b)) $$

Here a larger $\lambda$ gives a wider margin and a smaller $\lambda$ gives a narrow margin.
Basically, we can think of $\lambda$ as essentially equal to $1/C$. These parameters encode the
regularization strength, and they will need to be tuned with the other model parameters. 

As a final note, once we write the cost function in the implementation below, the combined
weight-intercept matrix $\mathbf{W}$ will be used instead of the explicit formula above:

$$ J(\mathbf{W}) = \frac{1}{2}||\mathbf{W}||^2 + C \left[ \frac{1}{n} \sum_{i=1}^{n} max \left(0, 1
- y_i \left(\mathbf{W} \cdot X_i\right)\right)\right] $$

#### Gradient Calculation


The gradient of the cost function is given by the following:

$$ \nabla_{\mathbf{W}} J(\mathbf{W}) = \frac{1}{n} \sum_{i=1}^n 
   \begin{cases}
    \mathbf{W} \qquad \text{if } max \left(0, 1 - y_i \left(\mathbf{W} \cdot X_i\right)\right) = 0\\
    \mathbf{W} - C y_i x_i \qquad \text{otherwise}.
   \end{cases} $$

The process for updating weights using gradient descent is the following:

1. Compute gradient: $\nabla_{\mathbf{W}} J(\mathbf{W})$
2. Update weights by moving opposite the gradient with learning rate $\alpha$: $\mathbf{W} =
   \mathbf{W} - \alpha \nabla_{\mathbf{W}} J(\mathbf{W})$ 
3. Repeat until, ideally, $J(W)$ is minimized.




#### Batch vs. Stochastic Gradient Descent


The above process can be implemented in multiple ways, most commonly by computing the gradient on
either the entire data set at once (known as Batch Gradient Descent (BCG)) or on smaller random
samplings of the data set (known as Stochastic Gradient Descent (SGD)). A great description of the
differences between the two approaches can be found at 

https://stats.stackexchange.com/questions/49528/batch-gradient-descent-versus-stochastic-gradient-descent

but let's give a brief description here.

BGD works well for geometrically "nice" error manifolds (the space on which we are minimizing the
function $J(\mathbf{W})$). In this case, global minimums are likely computed more efficiently due to
the simplicity of the gradient calculation on the error manifold. 

SGD, on the other hand, tends to work better on more complicated error manifolds. This is because
the gradient calculations only occur on a small sampling of the data, and the model is less likely
to get "stuck" in a local minimum, of which there may be many depending on the geometry of the error
manifold. The resulting process tends to also be much more efficient computationally, requiring less
memory on the system. 

BGD's computational expense and "niceness" assumptions make it appropriate for getting the idea of
the algorithm across, but it is not generally used in practice. SGD benefits from the statistical
stability that comes with averaging out repeated random samplings of a dataset and performing
simpler calculations in each pass. 

We will give an implementation of both below just to play around with these differences.

It will also help us to integrate a stoppage condition in order to prevent our SGD procedure from
overdoing it. The way we will do it is to first set a percentage improvement on the
cost-minimization process every $2^{n}th$ epoch which will be known as the cost threshold. The model
will continue to improve until it fails to improve by the cost threshold percentage, at which time
it will terminate and return the most up-to-date weights.

### SVM Class



In [7]:
# Importing necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
# For splitting test data sets:
from sklearn.model_selection import train_test_split

In [8]:
# Define the SVM class
class SVM():
    """This class gives an implementation of a support vector machine binary classifier.
    """

    def __init__(self, X, y, regularization=1000, learning_rate=0.01, 
                 max_epochs=5000, cost_thresh=0.01):
        """Initialize basic parameters and run a train-test-split, fit the model, and test on the
        test subset of the input data set.

        Args:
            X (2DArray): Input features (n x m matrix with n data points and m features)
            y (1DArray): Output classes (n x 1 matrix of classes associated to data)
            regularization (float, optional): regularization constant. Defaults to 1000.
            learning_rate (float, optional): learning rate for gradient descent. Defaults to 0.01.
            max_epochs (int, optional): number of epochs to iterate through. Defaults to 5000.
            cost_thresh (float, optional): Percentage threshold for cost minimization process. Defaults to 0.01.
        """
        self.features = X
        self.outputs = y
        self.regularization = regularization
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.cost_threshold = cost_thresh
        self.N = X.shape[0]

        # TODO: Add in normalization step for the input data

        # Add a column of 1's to the feature data since we combined the intercept calculation with
        # the rest of the weights
        self.features = np.concatenate((self.features, np.ones((self.shape[0], 1))), axis = 1)

        # Initialize weights
        self.weights = np.zeros(X.shape[1])

        # Train-Test-split
        self.X_train, self.y_train, self.X_test, self.y_test = train_test_split(X, y,
                                                                                shuffle=True,
                                                                                random_state=435,
                                                                                test_size=.25,
                                                                                stratify=y)

        # TODO: Update all of the functions below to take general X, y rather than the whole data
        # set. 

    def cost(self):
        N = self.features.shape[0]
        # distances from the hyperplane
        distances = 1 - self.outputs*(np.dot(self.weights, self.features))
        # filter out negative distances and convert to 0 (same as max function above)
        distances[distances < 0] = 0
        sum_loss = self.regularization* (np.sum(distances)/N)

        cost = 1 / 2 * np.dot(self.weights, self.weights) + sum_loss
        return cost

    # Define batch gradient, which can be used to compute the gradient on any size batch/sampling
    def cost_gradient_batch(self, X_bat, y_bat):
        # check if numbers or vectors are given (single item batches)
        if type(y_bat) == np.float_:
            y_bat = np.array([y_bat])
            X_bat = np.array([X_bat])
        
        # now we can run the same process for any size batch
        distance = 1 - (y_bat * np.dot(X_bat, self.weights))
        W_grad = np.zeros(len(self.weights)) # TODO: if W is passed as an array, may need to switch to shape[0]

        for ind, dist in enumerate(distance):
            if max(0, dist) == 0:
                dist_ind = self.weights
            else:
                dist_ind = self.weights - (self.regularization * y_bat[ind] * X_bat[ind])
            W_grad += dist_ind
        W_grad = W_grad/len(y_bat)

        return W_grad

    # Define the stochastic gradient descent function
    def sgd(self):

        # Set up stoppage condition params
        n_power = 0
        prev_cost = float('inf')

        for epoch in range(1, self.max_epochs):
            # shuffle data
            X_shuff, y_shuff = shuffle(self.features, self.outputs)
            # loop through and compute gradients for each datum
            for ind, x in enumerate(X_shuff):
                grad = self.cost_gradient_batch(x, self.outputs[ind])
                self.weights -= self.learning_rate*grad 

            # Check to see if we hit our stoppage condition
            if epoch == 2**(n_power) or epoch == self.max_epochs - 1:
                new_cost = self.cost()
                print("Epoch is:{} and Cost is: {}".format(epoch, new_cost))
                # check stoppage
                if abs(prev_cost - new_cost) < self.cost_threshold * prev_cost:
                    return self.weights
                prev_cost = new_cost
                n_power += 1

        return self.weights

In [11]:
X = np.array([[1,2],
             [3, 4]])
x = np.concatenate((X, np.ones((X.shape[0], 1))), axis = 1)
print(x)

[[1. 2. 1.]
 [3. 4. 1.]]
